In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import cufflinks as cf
import plotly
import plotly.graph_objs as go
import plotly.subplots as sp
# Connect plotly offline
plotly.offline.init_notebook_mode(connected=True)

cf.set_config_file(world_readable=False,theme='pearl')


display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
sns.set(font_scale=1.4)

In [3]:
start = dt.datetime(2005, 1, 1)
end = dt.datetime.now()
STOCK = 'VOO'

In [4]:
def cast_stock_df(df):
    df = df.copy(deep=True)
    return df.astype(float)

In [5]:
df = web.DataReader(
    STOCK,
    data_source='yahoo',
    start=start,
    end=end,
)
df = cast_stock_df(df)

In [6]:
df.shape

(2413, 6)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2413 entries, 2010-09-09 to 2020-04-09
Data columns (total 6 columns):
High         2413 non-null float64
Low          2413 non-null float64
Open         2413 non-null float64
Close        2413 non-null float64
Volume       2413 non-null float64
Adj Close    2413 non-null float64
dtypes: float64(6)
memory usage: 132.0 KB


In [8]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-09-09,102.500000,101.139999,102.500000,101.320000,26500.0,83.143929
2010-09-10,101.860001,101.300003,101.680000,101.779999,8600.0,83.521400
2010-09-13,103.139999,102.500000,102.959999,103.059998,33700.0,84.571770
2010-09-14,103.480003,102.379997,102.839996,103.040001,59400.0,84.555374
2010-09-15,103.379997,102.400002,102.620003,103.300003,9200.0,84.768715


## Data preprocessing

### Add rolling avg

In [9]:
MA_WIN = 30
MA_COL = f'ma{MA_WIN}'
df[MA_COL] = df['Adj Close'].rolling(window=MA_WIN, min_periods=0).median()

## Plots

In [10]:
fig = sp.make_subplots(rows=1, cols=2)
# Create scatter trace of text labels
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Adj Close'],
        mode="lines",
        name=f'All - Close',
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df[MA_COL],
        mode="lines",
        name=f'All - {MA_COL}',
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df[pd.Timestamp('2019-01-01'):].index,
        y=df[pd.Timestamp('2019-01-01'):]['Adj Close'],
        mode="lines",
        name=f'recent - Close',
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=df[pd.Timestamp('2019-01-01'):].index,
        y=df[pd.Timestamp('2019-01-01'):][MA_COL],
        mode="lines",
        name=f'recent - {MA_COL}',
    ),
    row=1,
    col=2,
)
fig.update_layout(height=500, title_text=f"Stock: {STOCK}")
fig.show()

### Use cufflinks to generate candlebar chart

In [33]:
cf.help()

Use 'cufflinks.help(figure)' to see the list of available parameters for the given figure.
Use 'DataFrame.iplot(kind=figure)' to plot the respective figure
Figures:
	bar
	box
	bubble
	bubble3d
	candle
	choroplet
	distplot
	heatmap
	histogram
	ohlc
	pie
	ratio
	scatter
	scatter3d
	scattergeo
	spread
	surface
	violin


In [34]:
cf.QuantFig??

In [38]:
qf = cf.QuantFig(
    df[pd.Timestamp('2019-01-01'):],
    title='VOO',
    legend='top',
    name='GS',
    up_color='green',
    down_color='red',
    rangeslider=True,
    rangeselector={ 'steps':['1y','2 months','ytd','reset'], 'bgcolor' : ('grey',.3), 'x': 0.8 , 'y' : 0.95},
)
# qf.add_bollinger_bands()
fig = qf.iplot(asFigure=True)
fig.show()